In [4]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [2]:
data = pd.read_csv("Datasetik.csv")
questions = data["Question"].tolist()
answers = data["Answer"].tolist()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(questions)
questions

['Какие качества нравятся в человеке?',
 'Любимый алкогольный напиток?',
 'Лучше все просчитывать или импровизировать?',
 'Какая твоя самая любимая вещь?',
 'Есть ли у тебя фобии?',
 'Твой самый эпичный промах в жизни?',
 'Какой твой любимый фильм и почему? ',
 'Какой твой любимый сериал и почему? ',
 'Какой твой любимый жанр музыки и почему? ',
 'Какой твой любимый писатель?',
 'Как хотел (-а) бы изменить мир?',
 'В чем не хочешь себе признаваться?',
 'Самая любимая игра?',
 'Твоя любимая песня и группа сейчас?',
 'Чего больше всего боишься в жизни?',
 'Кто из нас лучше одевается?',
 'Какая у тебя любимая одежда?',
 'Кем мечтал (-а) быть в детстве?',
 'Что вкуснее: кофе или чай?',
 'О чем можешь говорить часами?',
 'Что хочешь сделать сумасшедшего?',
 'Чем любишь заниматься в любимое время?',
 'Какая у тебя причина вставать по утрам?',
 'На кого ты похож?',
 'Ты больше любишь зиму или лето?',
 'Что до сих пор вызывает страх?',
 'Что больше всего раздражает?',
 'Любимый персонаж из мул

In [2]:
def get_best_answer(user_question):
       user_question_tfidf = vectorizer.transform([user_question])
       similarities = cosine_similarity(user_question_tfidf, tfidf_matrix)
       best_index = np.argmax(similarities)
       return answers[best_index]

In [4]:
while True:
    user_input = input("Enter your request (to exit, write 'stop'): ")
    best_answer = get_best_answer(user_input)
    print(best_answer)
    if user_input.lower() == 'stop':
        break

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2

def preprocess_input(user_input):
    nlp = spacy.load("ru_core_news_sm")
    stopwords_ru = stopwords.words('russian')
    morph = pymorphy2.MorphAnalyzer()

    
    result = [morph.parse(token.text.lower())[0].normal_form for token in nlp(user_input) if not token.is_punct and token.text.lower() not in stopwords_ru]
        
        

    return result

def vectorize_input(processed_input):
    from transformers import BertTokenizer, BertForSequenceClassification
    model_name = 'DeepPavlov/rubert-base-cased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    
    result = tokenizer.encode_plus(processed_input)['input_ids']

    return result


def postprocess_output(output_tensor):
    
    labels = answers[:]
    
    _, predicted_index = torch.max(output_tensor, dim=1)
   
    output_label = labels[predicted_index.item()]
    return output_label

class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(Transformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_dim, num_heads, hidden_dim, dropout),
            num_layers)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.linear(x)
        return x


num_epochs = 3 
batch_size = 1 


user_input = input("Enter your request: ")


processed_input = preprocess_input(user_input)
vectorized_input = vectorize_input(processed_input)


input_tensor = torch.tensor(vectorized_input).unsqueeze(0)
from math import ceil

input_dim = len(vectorized_input) 
hidden_dim = len(vectorized_input) 
output_dim = len(vectorized_input)
num_layers = 4 
num_heads = 1
dropout = 0.2 


model = Transformer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    
    input_tensor = torch.tensor(vectorized_input, dtype=torch.float).unsqueeze(0)
    output_tensor = model(input_tensor)
    
    optimizer.zero_grad()  
   
    output = postprocess_output(output_tensor)

    print("Answer: ", output)